In [1]:
import pandas as pd 
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer
import random
import torch

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tokenize_function(example):
    return (example)

df = pd.read_csv("erraticana-offtopic-raw.csv")
df = df[df["Author"] == "bonepriest#6318"]
bone_dataset = df[df["Content"].str.strip().str.contains(r"\s", regex=True, na=False) == True]
bone_dataset = bone_dataset.drop(['AuthorID', 'Author', 'Date', 'Attachments', 'Reactions'], axis=1)

In [3]:
bone_content = bone_dataset.Content.values.tolist()

In [4]:
random.shuffle(bone_content)

In [5]:
split_train_valid = 0.9

# split dataset
train_size = int(split_train_valid * len(bone_dataset))
valid_size = len(bone_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(bone_content, [train_size, valid_size])

In [6]:
def make_dataset(dataset, epochs):
    total_text = '<|endoftext|>'
    message = [t for t in dataset]
    for _ in range(epochs):
        random.shuffle(message)
        total_text += '<|endoftext|>'.join(message) + '<|endoftext|>'
    return total_text

In [8]:
EPOCHS = 4

with open('bone_train.txt', 'w', encoding='utf-8') as f:
    data = make_dataset(train_dataset, EPOCHS)
    f.write(data)

with open('bone_valid.txt', 'w', encoding='utf-8') as f:
    data = make_dataset(valid_dataset, 1)
    f.write(data)

In [9]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 16
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\USER/.netrc


True

In [16]:
import os
import subprocess

train_file_path = "bone_train.txt"
valid_file_path = "bone_valid.txt"  

command = [
    "python",
    "run_clm.py",
    "--output_dir=output/bone-gpt",
    "--overwrite_output_dir",
    "--overwrite_cache",
    "--model_type=gpt2",
    "--model_name_or_path=gpt2",
    "--do_train",
    f"--train_file={train_file_path}",
    "--do_eval",
    f"--validation_file={valid_file_path}",
    "--eval_steps=20",
    "--logging_steps=20",
    "--per_gpu_train_batch_size=1",
    "--num_train_epochs=1"
]

subprocess.run(command, check=True)

try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print(result.stdout)  # Output from the command
except subprocess.CalledProcessError as e:
    print(e.stderr)  # Error output from the command


CalledProcessError: Command '['python', 'run_clm.py', '--output_dir=output/bone-gpt', '--overwrite_output_dir', '--overwrite_cache', '--model_type=gpt2', '--model_name_or_path=gpt2', '--do_train', '--train_file=bone_train.txt', '--do_eval', '--validation_file=bone_valid.txt', '--eval_steps=20', '--logging_steps=20', '--per_gpu_train_batch_size=1', '--num_train_epochs=1']' returned non-zero exit status 1.

In [14]:
%env WANDB_PROJECT=bonegpt-dev

env: WANDB_PROJECT=bonegpt-dev


In [ ]:
SENTENCES = ["I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

In [ ]:
import random
seed = random.randint(0, 2**32-1)
seed

In [ ]:
examples = []
num_return_sequences = 5

for start in SENTENCES:
    val = !python run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/bone_gpt \
        --length 160 \
        --num_return_sequences $num_return_sequences \
        --temperature 1 \
        --p 0.95 \
        --seed $seed \
        --prompt {'"<|endoftext|>' + start + '"'}
    generated = [val[-1-2*k] for k in range(num_return_sequences)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')